# Transformation 

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType


### For best practices it's recoment to store those values inside Key Vaults   

In [0]:
# saving client id, tenant id and secret value inside of variables 

client_id = ""
tenant_id = ""
secret_value = ""

In [0]:
# Mount ADLS Gen2 or Blob Storage with ABFS

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com//oauth2/token"}


dbutils.fs.mount(
source = "abfss://tokyo-olympic-2021@tokyo2020ch.dfs.core.windows.net/",
mount_point = "/mnt/tokyolympic",
extra_configs = configs)

True

In [0]:
# checking files
%fs

ls "/mnt/tokyolympic/transformed"

In [0]:
# Creating dataframes for each


athletes = spark.read.format("csv").option("header", "True").option("inferSchema","true").load("/mnt/tokyoolymic2021/athletes.csv")
coaches = spark.read.format("csv").option("header", "True").option("inferSchema","true").load("/mnt/tokyoolymic2021/coaches.csv")
gender = spark.read.format("csv").option("header", "True").option("inferSchema","true").load("/mnt/tokyoolymic2021/gender.csv")
medals = spark.read.format("csv").option("header", "True").option("inferSchema","true").load("/mnt/tokyoolymic2021/medals.csv")
teams = spark.read.format("csv").option("header", "True").option("inferSchema","true").load("/mnt/tokyoolymic2021/teams.csv")

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
# checking Schemas 
athletes.printSchema()
coaches.printSchema()
gender.printSchema()
medals.printSchema()
teams.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)

root
 |-- Rank: integer (nullable = true)
 |-- Team/NOC: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)

root
 |-- Name: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
gender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

<h3>Transform the schema of gender from string to int</h3>

<p>
Inside the table Gender there are columns: Femele, Male, and Total are string type and the data storage in those columns are number in nature,
also this data can be use in mathematical operations. It's necessary to Transform the data type to ensure the data integrity  
</p>

In [0]:
gender = gender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team/NOC: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals
top_gold = medals.orderBy("Gold", ascending= False).select("Rank", "Team/NOC", "Gold").show()

+----+--------------------+----+
|Rank|            Team/NOC|Gold|
+----+--------------------+----+
|   1|United States of ...|  39|
|   2|People's Republic...|  38|
|   3|               Japan|  27|
|   4|       Great Britain|  22|
|   5|                 ROC|  20|
|   6|           Australia|  17|
|   7|         Netherlands|  10|
|   8|              France|  10|
|   9|             Germany|  10|
|  10|               Italy|  10|
|  14|                Cuba|   7|
|  13|         New Zealand|   7|
|  12|              Brazil|   7|
|  11|              Canada|   7|
|  15|             Hungary|   6|
|  16|   Republic of Korea|   6|
|  19|               Kenya|   4|
|  17|              Poland|   4|
|  18|      Czech Republic|   4|
|  20|              Norway|   4|
+----+--------------------+----+
only showing top 20 rows



### Writing inside of the container for transformed data

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header", "true").csv("/mnt/tokyolympic/transformed/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyolympic/transformed/coaches")
gender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyolympic/transformed/gender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyolympic/transformed/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyolympic/transformed/teams")